# Try Google Translate API

## Setup

### imports

In [7]:
import pandas as pd, numpy as np, seaborn as sns, mysql.connector as mysql
import os, re, sqlalchemy, datetime
import pickle

In [8]:
import mysql.connector as mysql
import sqlalchemy

In [15]:
import os, six
from google.cloud import translate_v2 as translate

### login

#### get credentials into notebook

In [10]:
mysql_user = 'bhrdwj'
# mysql_pass = input(f'Enter the MySQL password for user {mysql_user}: ')

#### connect to database

In [12]:
def connect_mariadb():
    host='localhost'; user=mysql_user; passwd=mysql_pass; dbname='jawiki';
    cxn = mysql.connect(host=host,user=user,passwd=passwd, database=dbname)
    cur = cxn.cursor()

    connection_str = 'mysql+mysqlconnector://'+user+':'+passwd+'@'+host+'/'+dbname  # removed this after host +':'+dbport
    try:
        engine = sqlalchemy.create_engine(connection_str)
        conn = engine.connect()
    except Exception as e:
        print('Database connection error - check creds')
        print(e)
    return cxn, cur, engine, conn
        
cxn, cur, engine, conn = connect_mariadb()

#### get ```tablenames```

In [14]:
metadata = sqlalchemy.MetaData(conn)
metadata.reflect()
current_tables = list(metadata.tables.keys())
current_tables;

## translate: ja2en_txt, ja2en_ser

### get gcloud credentials

In [16]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/home/bhrdwj/git/.my-translation-sa_keys.json'
bq_client = translate.Client()

### functions for translation

In [17]:
translate_client = translate.Client()

def ja2en_txt(el):
    """Translates text into the target language.
    Target must be an ISO 639-1 language code.
    See https://g.co/cloud/translate/v2/translate-reference#supported_languages
    """
    # import six
    # from google.cloud import translate_v2 as translate
    translate_client = translate.Client()
    
    if isinstance(el, six.binary_type):
        el = el.decode("utf-8")
    if not isinstance(el, str):
        return el
    
    text = el
    return translate_client.translate(
        text, target_language='EN', source_language='JA'
    )['translatedText']


def ja2en_ser(ser):
    """
    Maps each element of a series with ja2en_txt, only if the type is string
    """
    assert isinstance(ser,pd.Series)
    if pd.api.types.is_object_dtype(ser):
        return ser.map(lambda x: translate_client.translate(x, target_language='EN', source_language='JA')['translatedText'])    
    else:
        return ser

### test

#### test ja2en_txt

In [18]:
ja2en_txt('熱帯低気圧')

'Tropical cyclone'

#### test ja2en_ser

##### get series

In [20]:
jp_ser = pd.Series(
    ["こりおりのちから",
    "コリオリの力",
    "たいふう",
    "台風",
    "さいくろん",
    "サイクロン",
    "熱帯低気圧",
    "せきらんうん",
    "積乱雲",
    "はりけん",
    "ハリケーン",
    "ういりういり",
    "ウィリー・ウィリー",
    "たいふう",
    "台風",
    "ふしわらのこうか",
    "藤原の効果",
    "はりけん",
    "ハリケーン"
    ]
)

##### test

In [23]:
pd.concat((jp_ser.to_frame(), ja2en_ser(jp_ser).to_frame()), axis="columns")

,0,0
0,こりおりのちから,The power of Coriolis
1,コリオリの力,Coriolis force
2,たいふう,Typhoon
3,台風,Typhoon
4,さいくろん,Cyclone
5,サイクロン,Cyclone
6,熱帯低気圧,Tropical cyclone
7,せきらんうん,Cumulonimbus
8,積乱雲,Cumulonimbus
9,はりけん,Hariken


# END